In [43]:
import json
import requests
import yaml
import pandas as pd
from pandas import concat
from datetime import datetime
from pandas import json_normalize

In [79]:
### Get list of commune
# https://www.bfs.admin.ch/bfs/en/home/statistics/catalogues-databases/tables.assetdetail.16484444.html
commune_df = pd.read_csv("./../../data/raw/RegionalPortraits21.03.01.csv", sep=';', low_memory=False)
commune_df = commune_df[['REGION', 'VALUE']]

# for row in commune_df.iterrows():
#     region = row[1]['REGION'] # Region 
#     population = row[1]['VALUE'] # Population
#     print(f'Region = `{region}`, Population = `{population}`')
    
commune_df

,REGION,VALUE
0,Aeugst am Albis,1981
1,Affoltern am Albis,12303
2,Bonstetten,5572
3,Hausen am Albis,3751
4,Hedingen,3778
...,...,...
2167,Vendlincourt,559
2168,Basse-Allaine,1238
2169,Clos du Doubs,1263
2170,Haute-Ajoie,1085


In [55]:
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

# actual api key
config = yaml.load(open("./../../credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = config["google_map_key"]

# get method of requests module, return response object
row_num = 0
request_num = 0
for row in commune_df.iterrows():
    row_num += 1
    # if row_num in range(1, 207): # start from 1
    #     continue
    
    region = row[1]['REGION'].replace('/', '__') # Region | Commune
    
    # text string on which to search
    query = f'supermarkets near {region}, Switzerland'
    
    df = pd.DataFrame()
    next_page_token = None
    while True:
        request_num += 1
        req = requests.get(url + "query=" + query + "&key=" + gmaps_key \
           #+ "&region=ch&type=supermarket" \
           + ('' if next_page_token == None else f'&pageToken={next_page_token}'))

        # json method of response object: json format data -> python format data
        places_json = req.json()
        print(places_json)

        # process request response
        for result in places_json['results']:
            df = pd.concat([df, json_normalize(result)], ignore_index=True)
            #df.loc[df.astype(str).drop_duplicates().index] # otherwise will be TypeError: unhashable type: 'list'

        # inform an request has been processed
        # print(f"{i} - A request has been processed at", datetime.now().strftime("%H:%M:%S"))

        if ('next_page_token' in places_json.keys()):
            next_page_token = places_json['next_page_token']
        else:
            #df = df.loc[df.astype(str).drop_duplicates(subset=['place_id']).index]
            df = df.drop_duplicates(subset=['place_id'])
            df.to_csv(f"./../../data/raw/gmaps_data_{region}.csv")
            print(f"{row_num} - {request_num} - gmaps_data_{region}.csv created at", datetime.now().strftime("%H:%M:%S"))
            break
print('done!')

{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
1 - 1 - gmaps_data_Aeugst am Albis.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
2 - 2 - gmaps_data_Affoltern am Albis.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
3 - 3 - gmaps_data_Bonstetten.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
4 - 4 - gmaps_data_Hausen am Albis.csv created at 20:51:17
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
5 - 5 - gmaps_data_Hedingen.csv created at 20:51:17
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 

KeyboardInterrupt: 

In [41]:
### Merging all data of Commune into one raw table
df = pd.DataFrame()
for row in commune_df.iterrows():
    region = row[1]['REGION'].replace('/', '__') # Region | Commune
    df = pd.concat([df, pd.read_csv(f"./../../data/raw/gmaps_data_{region}.csv", low_memory=False)])
df = df.drop_duplicates(subset=['place_id'])
df.to_csv("./../../data/raw/gmaps_data.csv")
df

,Unnamed: 0,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,...,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,price_level,permanently_closed
0,0,OPERATIONAL,"Allmendstrasse 5, 8914 Aeugst am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Volg,"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJoetcG74HkEcR5_9mPkmNtSI,4.4,...,8.486222,47.269277,8.487494,47.266577,8.484794,True,7F9P+6C Aeugst am Albis,8FVC7F9P+6C,NaN,NaN
1,1,OPERATIONAL,"Oberdorfstrasse 4, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Migros Supermarkt,"[{'height': 1920, 'html_attributions': ['<a hr...",ChIJiUOy2zkGkEcRy22nDwkqAks,4.2,...,8.453773,47.279066,8.455123,47.276367,8.452424,True,7FH3+4G Affoltern am Albis,8FVC7FH3+4G,NaN,NaN
2,2,OPERATIONAL,"Büelstrasse 15, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Coop Supermarkt Affoltern am Albis,"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJk6WFSkcGkEcRwsvlTASqby8,4.1,...,8.445676,47.275559,8.447437,47.272859,8.444737,True,7CFW+G7 Affoltern am Albis,8FVC7CFW+G7,NaN,NaN
3,3,OPERATIONAL,"Centralweg 3, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 1200, 'html_attributions': ['<a hr...",ChIJW4AoZTcGkEcRZ-7-G4qYMg0,4.1,...,8.453030,47.279424,8.454497,47.276724,8.451797,True,7FH3+66 Affoltern am Albis,8FVC7FH3+66,NaN,NaN
4,4,OPERATIONAL,"Industriestrasse 5, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,ALDI SUISSE,"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJLzM1Oh8HkEcRwBmfpSnSZx8,4.1,...,8.447015,47.274182,8.448102,47.271483,8.445402,True,7CFW+3R Affoltern am Albis,8FVC7CFW+3R,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,6,OPERATIONAL,"Dorfstrasse 1, 8197 Rafz",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Volg Rafz,"[{'height': 1728, 'html_attributions': ['<a hr...",ChIJadCMQuZ5kEcRlMSlbbmuFzg,4.4,...,8.536855,47.613674,8.538179,47.610974,8.535479,True,JG6P+WM Rafz,8FVCJG6P+WM,NaN,NaN
12,12,OPERATIONAL,"Industriestraße 36, 79807 Lottstetten, Germany",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl,"[{'height': 2988, 'html_attributions': ['<a hr...",ChIJoV16IE54kEcRwf420aRhKKc,4.1,...,8.578581,47.632467,8.580241,47.629767,8.577541,True,"JHJH+CC Lottstetten, Germany",8FVCJHJH+CC,NaN,NaN
14,14,OPERATIONAL,"Emmerstrasse 12, 8192 Glattfelden",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Coop Supermarkt Glattfelden,"[{'height': 1960, 'html_attributions': ['<a hr...",ChIJYQBygs12kEcRFEG25WHmtks,4.1,...,8.502778,47.558039,8.504261,47.555339,8.501562,True,HG43+P4 Glattfelden,8FVCHG43+P4,NaN,NaN
15,15,OPERATIONAL,"Schaffhauserstrasse 39, 8193 Eglisau",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,migrolino Eglisau,"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJPSOMLDF3kEcR3XUwsuQU9Hs,4.3,...,8.516040,47.582226,8.517641,47.579526,8.514942,True,HGJ8+8C Eglisau,8FVCHGJ8+8C,NaN,NaN


In [72]:
original_df = pd.read_csv("./../../data/raw/gmaps_data.csv", low_memory=False)
df = original_df[['place_id', 'formatted_address', 'name', 'rating', 'types', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng', 'plus_code.compound_code']].copy()

df['canton'] = df.apply(lambda row: row.formatted_address.rsplit(', ', 2)[1][5:], axis=1)
df['brand'] = df['name'].apply(lambda x: 'Migros' if ('Migros' in x) or ('migros' in x) else x)
df.to_csv("./../../data/processed/gmaps_data_all_brands.csv")
df[df['brand'] == 'Migros'].to_csv("./../../data/processed/gmaps_data_migros_only.csv")
df[df['brand'] != 'Migros'].to_csv("./../../data/processed/gmaps_data_other_brands.csv")
df

,place_id,formatted_address,name,rating,types,user_ratings_total,geometry.location.lat,geometry.location.lng,plus_code.compound_code,canton,brand
0,ChIJoetcG74HkEcR5_9mPkmNtSI,"Allmendstrasse 5, 8914 Aeugst am Albis",Volg,4.4,"['supermarket', 'grocery_or_supermarket', 'sto...",47.0,47.267973,8.486222,7F9P+6C Aeugst am Albis,Aeugst am Albis,Volg
1,ChIJiUOy2zkGkEcRy22nDwkqAks,"Oberdorfstrasse 4, 8910 Affoltern am Albis",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'sto...",578.0,47.277794,8.453773,7FH3+4G Affoltern am Albis,Affoltern am Albis,Migros
2,ChIJk6WFSkcGkEcRwsvlTASqby8,"Büelstrasse 15, 8910 Affoltern am Albis",Coop Supermarkt Affoltern am Albis,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",1179.0,47.273871,8.445676,7CFW+G7 Affoltern am Albis,Affoltern am Albis,Coop Supermarkt Affoltern am Albis
3,ChIJW4AoZTcGkEcRZ-7-G4qYMg0,"Centralweg 3, 8910 Affoltern am Albis",Lidl Schweiz,4.1,"['supermarket', 'convenience_store', 'grocery_...",292.0,47.278007,8.453030,7FH3+66 Affoltern am Albis,Affoltern am Albis,Lidl Schweiz
4,ChIJLzM1Oh8HkEcRwBmfpSnSZx8,"Industriestrasse 5, 8910 Affoltern am Albis",ALDI SUISSE,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",292.0,47.272725,8.447015,7CFW+3R Affoltern am Albis,Affoltern am Albis,ALDI SUISSE
...,...,...,...,...,...,...,...,...,...,...,...
159,ChIJadCMQuZ5kEcRlMSlbbmuFzg,"Dorfstrasse 1, 8197 Rafz",Volg Rafz,4.4,"['supermarket', 'grocery_or_supermarket', 'sto...",146.0,47.612332,8.536855,JG6P+WM Rafz,Rafz,Volg Rafz
160,ChIJoV16IE54kEcRwf420aRhKKc,"Industriestraße 36, 79807 Lottstetten, Germany",Lidl,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",1019.0,47.631005,8.578581,"JHJH+CC Lottstetten, Germany",7 Lottstetten,Lidl
161,ChIJYQBygs12kEcRFEG25WHmtks,"Emmerstrasse 12, 8192 Glattfelden",Coop Supermarkt Glattfelden,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",248.0,47.556828,8.502778,HG43+P4 Glattfelden,Glattfelden,Coop Supermarkt Glattfelden
162,ChIJPSOMLDF3kEcR3XUwsuQU9Hs,"Schaffhauserstrasse 39, 8193 Eglisau",migrolino Eglisau,4.3,"['supermarket', 'grocery_or_supermarket', 'sto...",123.0,47.580756,8.516040,HGJ8+8C Eglisau,Eglisau,migrolino Eglisau


In [83]:
### Process supermarkets_new.csv

df = pd.read_csv("./../../data/raw/supermarkets_new.csv", low_memory=False)
df = df.drop_duplicates(subset=['place_id'])
df['commune'] = df.apply(lambda row: row.formatted_address.rsplit(', ', 2)[1][5:], axis=1)
df['brand'] = df['name'].apply(lambda x: 'Migros' if ('Migros' in x) or ('migros' in x) else x)
df.to_csv("./../../data/processed/supermarkets_new.csv")
df

,Unnamed: 0,formatted_address,name,place_id,rating,user_ratings_total,geometry.location.lat,geometry.location.lng,opening_hours.open_now,permanently_closed,Street,commune,country,store_name,brand
0,3,"Obfelderstrasse 1, 8910 Affoltern am Albis, Sw...",migrolino Affoltern am Albis,ChIJURZXD2oGkEcRtXYXRHI1aFc,4.1,67.0,47.274687,8.445785,True,NaN,Obfelderstrasse 1,Affoltern am Albis,Switzerland,migro,migrolino Affoltern am Albis
1,4,"Büelstrasse 15, 8910 Affoltern am Albis, Switz...",Coop Supermarkt Affoltern am Albis,ChIJk6WFSkcGkEcRwsvlTASqby8,4.1,1179.0,47.273871,8.445676,True,NaN,Büelstrasse 15,Affoltern am Albis,Switzerland,coop,Coop Supermarkt Affoltern am Albis
2,5,"Centralweg 3, 8910 Affoltern am Albis, Switzer...",Lidl Schweiz,ChIJW4AoZTcGkEcRZ-7-G4qYMg0,4.1,292.0,47.278007,8.453030,True,NaN,Centralweg 3,Affoltern am Albis,Switzerland,lidl,Lidl Schweiz
3,6,"Oberdorfstrasse 4, 8910 Affoltern am Albis, Sw...",Migros Supermarkt,ChIJiUOy2zkGkEcRy22nDwkqAks,4.2,578.0,47.277794,8.453773,True,NaN,Oberdorfstrasse 4,Affoltern am Albis,Switzerland,migro,Migros
4,8,"Industriestrasse 5, 8910 Affoltern am Albis, S...",ALDI SUISSE,ChIJLzM1Oh8HkEcRwBmfpSnSZx8,4.1,292.0,47.272725,8.447015,True,NaN,Industriestrasse 5,Affoltern am Albis,Switzerland,aldi,ALDI SUISSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,10718,"Rte de Porrentruy 9, 2942 Alle, Switzerland",Coop Supermarché Coop Alle,ChIJOzrYYrfxkUcRtq93KDCgFm4,4.1,129.0,47.425800,7.124070,True,NaN,Rte de Porrentruy 9,Alle,Switzerland,coop,Coop Supermarché Coop Alle
2740,10722,"Rue de la Pierre-Percée 2, 2950 Courgenay, Swi...",Société coopérative de fromagerie de Courgenay...,ChIJ9UQKy8nwkUcRw0-Jl7-a__E,0.0,0.0,47.405915,7.119841,NaN,True,Rue de la Pierre-Percée 2,Courgenay,Switzerland,coop,Société coopérative de fromagerie de Courgenay...
2741,10746,"Rte des Rangiers 56, 2882 Saint-Ursanne, Switz...",Coopérative Et Société Agricole,ChIJx8GLkY36kUcRXndn4ed490E,0.0,0.0,47.366454,7.159582,NaN,NaN,Rte des Rangiers 56,Saint-Ursanne,Switzerland,coop,Coopérative Et Société Agricole
2742,10762,"Rte de Courgenay 18, 2900 Porrentruy, Switzerland",Coop Pronto avec station-service Porrentruy,ChIJoRZ5Txv3kUcRZEvMvkyOcNw,4.2,78.0,47.415821,7.079681,True,NaN,Rte de Courgenay 18,Porrentruy,Switzerland,coop,Coop Pronto avec station-service Porrentruy


In [82]:
### Add population from `commune_df` into `df`

df

,Unnamed: 0,formatted_address,name,place_id,rating,user_ratings_total,geometry.location.lat,geometry.location.lng,opening_hours.open_now,permanently_closed,Street,commune,country,store_name,brand
0,3,"Obfelderstrasse 1, 8910 Affoltern am Albis, Sw...",migrolino Affoltern am Albis,ChIJURZXD2oGkEcRtXYXRHI1aFc,4.1,67.0,47.274687,8.445785,True,NaN,Obfelderstrasse 1,Affoltern am Albis,Switzerland,migro,migrolino Affoltern am Albis
1,4,"Büelstrasse 15, 8910 Affoltern am Albis, Switz...",Coop Supermarkt Affoltern am Albis,ChIJk6WFSkcGkEcRwsvlTASqby8,4.1,1179.0,47.273871,8.445676,True,NaN,Büelstrasse 15,Affoltern am Albis,Switzerland,coop,Coop Supermarkt Affoltern am Albis
2,5,"Centralweg 3, 8910 Affoltern am Albis, Switzer...",Lidl Schweiz,ChIJW4AoZTcGkEcRZ-7-G4qYMg0,4.1,292.0,47.278007,8.453030,True,NaN,Centralweg 3,Affoltern am Albis,Switzerland,lidl,Lidl Schweiz
3,6,"Oberdorfstrasse 4, 8910 Affoltern am Albis, Sw...",Migros Supermarkt,ChIJiUOy2zkGkEcRy22nDwkqAks,4.2,578.0,47.277794,8.453773,True,NaN,Oberdorfstrasse 4,Affoltern am Albis,Switzerland,migro,Migros
4,8,"Industriestrasse 5, 8910 Affoltern am Albis, S...",ALDI SUISSE,ChIJLzM1Oh8HkEcRwBmfpSnSZx8,4.1,292.0,47.272725,8.447015,True,NaN,Industriestrasse 5,Affoltern am Albis,Switzerland,aldi,ALDI SUISSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,10718,"Rte de Porrentruy 9, 2942 Alle, Switzerland",Coop Supermarché Coop Alle,ChIJOzrYYrfxkUcRtq93KDCgFm4,4.1,129.0,47.425800,7.124070,True,NaN,Rte de Porrentruy 9,Alle,Switzerland,coop,Coop Supermarché Coop Alle
2740,10722,"Rue de la Pierre-Percée 2, 2950 Courgenay, Swi...",Société coopérative de fromagerie de Courgenay...,ChIJ9UQKy8nwkUcRw0-Jl7-a__E,0.0,0.0,47.405915,7.119841,NaN,True,Rue de la Pierre-Percée 2,Courgenay,Switzerland,coop,Société coopérative de fromagerie de Courgenay...
2741,10746,"Rte des Rangiers 56, 2882 Saint-Ursanne, Switz...",Coopérative Et Société Agricole,ChIJx8GLkY36kUcRXndn4ed490E,0.0,0.0,47.366454,7.159582,NaN,NaN,Rte des Rangiers 56,Saint-Ursanne,Switzerland,coop,Coopérative Et Société Agricole
2742,10762,"Rte de Courgenay 18, 2900 Porrentruy, Switzerland",Coop Pronto avec station-service Porrentruy,ChIJoRZ5Txv3kUcRZEvMvkyOcNw,4.2,78.0,47.415821,7.079681,True,NaN,Rte de Courgenay 18,Porrentruy,Switzerland,coop,Coop Pronto avec station-service Porrentruy


In [74]:
grouped_df = df.groupby(by=['commune', 'brand']).agg({
    'rating': ['mean', 'median', 'count'],
    'user_ratings_total': 'sum',
    'geometry.location.lat': 'count'
}).rename(columns={
    'rating_count': 'count'
}).reset_index()

grouped_df[grouped_df['brand'] == 'Migros']
grouped_df

#df.groupby(["item", "color"])["id"].count().reset_index(name="count")

commune                                   brand rating               \
                                                        mean median count   
0                                              Migros    4.4    4.4     1   
1      Aadorf                             ALDI SUISSE    4.3    4.3     1   
2      Aadorf  Coop Pronto Shop mit Tankstelle Aadorf    4.3    4.3     1   
3      Aadorf                  Coop Supermarkt Aadorf    4.3    4.3     1   
4      Aadorf                                  Migros    4.1    4.1     1   
...       ...                                     ...    ...    ...   ...   
2422  zerland                             Coop Outlet    4.2    4.2     1   
2423  zerland                             Coop Pronto    0.0    0.0     1   
2424  zerland        LANDI Lucens Société coopérative    5.0    5.0     1   
2425  zerland   La Butega da Munt Società Cooperativa    4.5    4.5     1   
2426  zerland   Società Cooperativa di Consumo Meride    0.0    0.0     1   

     user_ratings_total geometry.location.lat  
                    sum                 count  
0                 787.0                     1  
1                 481.0                     1  
2                 293.0                     1  
3                 320.0                     1  
4                 258.0                     1  
...                 ...                   ...  
2422              163.0                     1  
2423                0.0                     1  
2424                1.0                     1  
2425                4.0                     1  
2426                0.0                     1  

[2427 rows x 7 columns]

In [77]:
my_str = 'Airport Center, Prime Center 1, 8060 Zürich, Switzerland'

print(my_str.rsplit(', '))  # 👉️ ['one', 'two', 'three']
print(my_str.rsplit(', ', 2)[1][5:])

['Airport Center', 'Prime Center 1', '8060 Zürich', 'Switzerland']
Zürich
